In [1]:
# Dependencies
import cobra
import symengine
from copy import deepcopy
from itertools import chain
from cobra.util import solver
from random import shuffle
from  warnings import filterwarnings
from cobra.medium import minimal_medium
from ecology import *

# Create community model object
def initialize_community(name='none'):
    community = cobra.Model()
    community.id = 'microbial community'
    community.member_objs = set()
    community.members = []
    community.met_compartments = {}
    community.name = name
    return community


# Function to add reactions to consortia super-organism
def grow_community(community, member, minMedia=False):
    
    # Save new objective ID
    if len(member.name) > 0:
        member_name = member.name.replace(' ','')
        member_name = member_name.replace('_','')
    else:
        member_name = 'member' + str(len(community.objectives))
    obj = str(member.objective.expression).split()[0].split('*')[1]
    community.member_objs |= set([obj + '_' + member_name])
    obj_mets = [x.id + '_' + member_name for x in member.reactions.get_by_id(obj).metabolites]
    community.members.append(member_name)
    member_compartment = member_name + '_cytosol'
    
    # Define minimal medium
    if minMedia == True:
        min_rxns = _find_essential_metabolites(member)
        for x in member.exchanges:
            if x.id in min_rxns:
                continue
            else:
                x.lower_bound = 0.
    else:
        min_rxns = []
    
    # Extend exchange reactions
    if len(community.reactions) > 0:
        community_exch = set([x.id for x in community.exchanges])
    else:
        community_exch = []
    member_exch = set()
    member_extracellular = set()
    add_exch = []
    for rxn in member.exchanges:
        member_exch |= set([rxn.id])
        member_extracellular |= set([list(rxn.metabolites)[0].id])
        if rxn.id in min_rxns and rxn.id in community_exch:
            community.reactions.get_by_id(rxn.id).lower_bound = -1000.
        if rxn.id in community_exch:
            continue
        else:
            add_exch.append(deepcopy(rxn))
    community.add_reactions(add_exch)

    # Parse intracellular reaction
    member_copy = deepcopy(member)
    for rxn in member_copy.reactions:
        # check for exchange
        if rxn.id in member_exch:
            continue
        else:
            rxn.id = rxn.id + '_' + member_name
    member_copy.repair()
    member_intracellular = set()
    for cpd in member_copy.metabolites:
        # check for extracellular
        if cpd.id in member_extracellular:
            pass
        else:
            cpd.compartment = member_compartment
            cpd.id = cpd.id + '_' + member_name
            if not cpd.id in obj_mets:
                member_intracellular |= set([cpd.id])
    member_copy.repair()
                
    # Extend community model
    new_rxns = []
    for rxn in member_copy.reactions:
        if not rxn.id in member_exch:
            new_rxn = deepcopy(rxn)
            new_rxn.source_organism = member_name
            new_rxns.append(new_rxn)
    community.add_reactions(new_rxns)
        
    # Update community compartment metabolite membership
    consortia_model.met_compartments[member_compartment] = member_intracellular
    
    # Apply minimum growth constraint
    member_flux = member.slim_optimize()
    member_obj = str(member.objective.expression).split()[0].split('*')[1] + '_' + member_name
    member_min_constraint = community.problem.Constraint(community.reactions.get_by_id(member_obj).flux_expression, 
                                                lb=member_flux * 0.01)
    community.add_cons_vars(member_min_constraint)
    print('Added', member_name, 'compartment')
    return community


# Define minimal medium components
def _find_essential_metabolites(model, fraction=0.01):
    min_growth = model.slim_optimize() * fraction
    filterwarnings('ignore')
    minOut = minimal_medium(model, min_objective_value=min_growth, minimize_components=True)
    filterwarnings('default')
    rxns = set(list(minOut.index))
    return(rxns)


# Find transport reaction IDs
def _get_transporters(community):
    transporters = []
    exchs = set([x.id for x in community.exchanges])
    for rxn in community.reactions:
        if rxn.id in exchs: 
            continue
        else:
            compartments = set()
            for x in list(rxn.metabolites): compartments |= set([x.compartment])
            if len(compartments) > 1: transporters.append(rxn.id)
    
    return set(transporters)


def expand_transporters(community, fraction = 0):
    transporters = _get_transporters(community)
    transported = set()
    for x in transporters:
        transported |= set([y.id for y in community.reactions.get_by_id(x).metabolites])
    
    new_exchs = []
    new_rxns = []
    curr_new_rxn = 0
    for member in community.met_compartments.keys():
        for cpd in community.met_compartments[member]:
            if cpd in transported:
                continue
            else:
                cyt_cpd = community.metabolites.get_by_id(cpd)
                ext_id = cpd.split('_')[0] + '_e'
                try:
                    ext_cpd = community.metabolites.get_by_id(ext_id)
                except:
                    ext_cpd = deepcopy(cyt_cpd)
                    ext_cpd.compartment = 'extracellular'
                    ext_cpd.id = cpd.split('_')[0] + '_e'
                    community.add_boundary(ext_cpd, type='exchange', reaction_id='EX_'+ext_cpd.id, lb=0., ub=1000.)
                    new_exchs.append('EX_'+ext_cpd.id)
                
                # Add new transporters
                curr_new_rxn += 1
                new_rxn = cobra.Reaction('transporter_' + str(curr_new_rxn))
                new_rxn.name = ext_cpd.name + ' diffusion'
                new_rxn.lower_bound = -1000.
                new_rxn.upper_bound = 1000.
                new_rxn.add_metabolites({ext_cpd: -1.0, cyt_cpd: 1.0})
                new_rxns.append(new_rxn)
    
    # Open indicated fraction of new exchanges
    if fraction != 0:
        shuffle(new_exchs)
        perc_changed = round(len(new_exchs)*fraction)
        for x in new_exchs[0:perc_changed]: community.reactions.get_by_id(x).lower_bound = -1000.
    
    total_added = len(new_rxns) + len(new_exchs)
    print('Total added reactions:', total_added)
    if fraction != 0: print('Opened exchanges:', perc_changed)
    community.add_reactions(new_rxns)
    return community


# Constrain media consitions and member transport rates based on relative abundance
def constrain_media_and_abundance(community, media, abundances):
    
    # Set media conditions
    for exch in media.keys():
        community.reactions.get_by_id(exch).bounds = (float(media[exch]), 1000.)
    
    # Constrain uptake by species abundance
    transporters = _get_transporters(community)
    for rxn in transporters:
        abund_factor = abundances[community.reactions.get_by_id(rxn).source_organism]
        new_lb = community.reactions.get_by_id(rxn).lower_bound * abund_factor
        new_ub = community.reactions.get_by_id(rxn).upper_bound * abund_factor
        community.reactions.get_by_id(rxn).bounds = (new_lb, new_ub)
    
    return community
    

# Create composite community biomass objective
def solve_community_objective(community, obj_type='growth', fraction=0.5):
    
    # Growth maximization
    if obj_type == 'growth':
        community_flux_expression = symengine.RealDouble(0)
        for obj in community.member_objs:
            community_flux_expression += community.reactions.get_by_id(obj).flux_expression
        community_objective = community.problem.Objective(community_flux_expression * fraction, direction='max')
        community.objective = community_objective

    # Maximum parsimony
    rxn_vars = ((rxn.forward_variable, rxn.reverse_variable) for rxn in community.reactions)
    rxn_vars = chain(*rxn_vars)
    community.objective = community.problem.Objective(
        symengine.RealDouble(0), direction='min', sloppy=True, name='community_pfba_obj')
    community.objective.set_linear_coefficients({v: 1.0 for v in rxn_vars})
    community.solver.update()
    
    # Solve
    community_solution = community.optimize(objective_sense=None)    
    return community_solution


Keyring is skipped due to an exception: not all arguments converted during string formatting


In [2]:
mag_genre_dir = '/home/matt/Desktop/FMT_large_files/MAG_blast_results/reconstructions/final_genres/'
Erectale = cobra.io.read_sbml_model(mag_genre_dir+'super/iEr748.sbml')
#Afinegoldii = cobra.io.read_sbml_model(super_genres + 'iAfinegoldii.sbml')
#PspCT06 = cobra.io.read_sbml_model(super_genres + 'iPspCT06.sbml')
#Bvulgatus = cobra.io.read_sbml_model(super_genres + 'iBvulgatus.sbml')
#Ahadrus = cobra.io.read_sbml_model(super_genres + 'iAhadrus.sbml')
#Fprausnitzii = cobra.io.read_sbml_model(super_genres + 'iFprausnitzii.sbml')
#CspART55 = cobra.io.read_sbml_model(super_genres + 'iCspART55.sbml')
#Bdorei = cobra.io.read_sbml_model(super_genres + 'iBdorei.sbml')
Bpseudocatenulatum = cobra.io.read_sbml_model(mag_genre_dir+'super/iBp118.sbml')
#Osplanchnicus = cobra.io.read_sbml_model(super_genres + 'iOsplanchnicus.sbml')
#Eeligens = cobra.io.read_sbml_model(super_genres + 'iEeligens.sbml')
#Pdistasonis = cobra.io.read_sbml_model(super_genres + 'iPdistasonis.sbml')
#Dpiger = cobra.io.read_sbml_model(super_genres + 'iDpiger.sbml')
Bproducta = cobra.io.read_sbml_model(mag_genre_dir+'super/iBp829.sbml')

In [3]:
Erectale

Name,iEr748
Memory address,0x07fc48c4ffcd0
Number of metabolites,874
Number of reactions,871
Number of groups,0
Objective expression,1.0*biomass_GmPos - 1.0*biomass_GmPos_reverse_ba263
Compartments,"cytosol, extracellular"


In [4]:
Bpseudocatenulatum

Name,iBp118
Memory address,0x07fc48c4845b0
Number of metabolites,545
Number of reactions,499
Number of groups,0
Objective expression,1.0*biomass_GmPos - 1.0*biomass_GmPos_reverse_ba263
Compartments,"cytosol, extracellular"


In [5]:
Bproducta

Name,iBp829
Memory address,0x07fc452c33250
Number of metabolites,1045
Number of reactions,1021
Number of groups,0
Objective expression,1.0*biomass_GmPos - 1.0*biomass_GmPos_reverse_ba263
Compartments,"cytosol, extracellular"


In [6]:
consortia_model = initialize_community()

In [7]:
consortia_model = grow_community(consortia_model, Erectale)
consortia_model = grow_community(consortia_model, Bpseudocatenulatum)
consortia_model = grow_community(consortia_model, Bproducta)

Added Eubacteriumrectale compartment
Added Bifidobacteriumpseudocatenulatum compartment
Added Blautiaproducta compartment


In [8]:
#consortia_model = expand_transporters(consortia_model)

In [9]:
consortia_solution = solve_community_objective(consortia_model)

In [8]:
consortia_model

Name,microbial community
Memory address,0x07fc48c5125e0
Number of metabolites,2393
Number of reactions,2320
Number of groups,0
Objective expression,0.0
Compartments,"extracellular, Eubacteriumrectale_cytosol, Bifidobacteriumpseudocatenulatum_cytosol, Blautiaproducta_cytosol"


In [9]:
iCdR703 = cobra.io.load_json_model('/home/matt/Desktop/repos/Jenior_CdifficileGENRE_2021/data/reconstructions/iCdR703.json')

In [10]:
# Set reciprocal media conditions
min_growth = iCdR703.slim_optimize() * 0.01
minOut = minimal_medium(iCdR703, min_objective_value=min_growth, minimize_components=True)
cdf_rxns = set(list(minOut.index))
for x in iCdR703.exchanges:
    if x.id in cdf_rxns:
        x.lower_bound = -1000.
    else:
        x.lower_bound = 0.
        
for x in consortia_model.exchanges:
    try:
        iCdR703.reactions.get_by_id(x).bounds = x.bounds
    except:
        continue
        
for x in cdf_rxns:
    try:
        consortia_model.reactions.get_by_id(x).lower_bound = -1000.
    except:
        continue

In [11]:
# Create pFBA objective
objVal = iCdR703.slim_optimize()
obj_flux = iCdR703.problem.Constraint(iCdR703.reactions.biomass.flux_expression, lb=objVal * 0.5, ub=objVal)
iCdR703.add_cons_vars(obj_flux)

rxn_vars = ((rxn.forward_variable, rxn.reverse_variable) for rxn in iCdR703.reactions)
rxn_vars = chain(*rxn_vars)
iCdR703.objective = iCdR703.problem.Objective(
    symengine.RealDouble(0), direction='min', sloppy=True, name='pfba_obj')
iCdR703.objective.set_linear_coefficients({v: 1.0 for v in rxn_vars})
iCdR703.solver.update()
iCdR703_solution = iCdR703.optimize()

In [15]:
iCdR703_fluxes = iCdR703_solution.fluxes.to_frame()
iCdR703_fluxes['reaction'] = list(iCdR703_fluxes.index)
iCdR703_fluxes['name'] = [iCdR703.reactions.get_by_id(x).name for x in list(iCdR703_fluxes.index)]
iCdR703_exch = [x.id for x in iCdR703.exchanges]
boolean_series = iCdR703_fluxes.reaction.isin(iCdR703_exch)
iCdR703_fluxes = iCdR703_fluxes[boolean_series]
iCdR703_fluxes = iCdR703_fluxes[iCdR703_fluxes['fluxes'] < 0.0]

In [16]:
iCdR703_fluxes

,fluxes,reaction,name
EX_cpd00023_e,-1.555138e+01,EX_cpd00023_e,L-Glutamate exchange
EX_cpd00971_e,-7.983257e-02,EX_cpd00971_e,Na+ exchange
EX_cpd00051_e,-5.109285e+00,EX_cpd00051_e,L-Arginine exchange
EX_cpd00064_e,-4.404788e+01,EX_cpd00064_e,Ornithine exchange
EX_cpd10515_e,-7.983257e-02,EX_cpd10515_e,Fe2+ exchange
EX_cpd00035_e,-4.115893e+02,EX_cpd00035_e,L-Alanine exchange
EX_cpd00220_e,-8.781583e-02,EX_cpd00220_e,Riboflavin exchange
EX_cpd00117_e,-8.382420e+01,EX_cpd00117_e,D-Alanine exchange
EX_cpd00254_e,-7.983257e-02,EX_cpd00254_e,Mg exchange
EX_cpd00205_e,-7.983257e-02,EX_cpd00205_e,K+ exchange


In [17]:
consortia_fluxes = consortia_solution.fluxes.to_frame()
consortia_fluxes['reaction'] = list(consortia_fluxes.index)
consortia_fluxes['name'] = [consortia_model.reactions.get_by_id(x).name for x in list(consortia_fluxes.index)]
consortia_exch = [x.id for x in consortia_model.exchanges]
boolean_series = consortia_fluxes.reaction.isin(consortia_exch)
consortia_fluxes = consortia_fluxes[boolean_series]
consortia_fluxes = consortia_fluxes[consortia_fluxes['fluxes'] < 0.0]

In [18]:
consortia_fluxes

,fluxes,reaction,name
EX_cpd17005_e,-0.006920,EX_cpd17005_e,Co3+ exchange
EX_cpd00971_e,-0.016920,EX_cpd00971_e,Na+ exchange
EX_cpd00119_e,-0.010000,EX_cpd00119_e,L-Histidine exchange
EX_cpd00254_e,-0.016920,EX_cpd00254_e,Mg exchange
EX_cpd01311_e,-0.016920,EX_cpd01311_e,Dethiobiotin exchange
EX_cpd10515_e,-0.016920,EX_cpd10515_e,Fe+2 exchange
EX_cpd00205_e,-0.016920,EX_cpd00205_e,K+ exchange
EX_cpd00056_e,-0.016920,EX_cpd00056_e,Tpp exchange
EX_cpd00063_e,-0.016920,EX_cpd00063_e,Ca2+ exchange
EX_cpd00099_e,-0.010000,EX_cpd00099_e,Cl- exchange


In [12]:
consortia_CdR20291_competition = competition(consortia_model, iCdR703)

Starting flux none: 0.0 Clostridium difficile R20291: 672.14


/home/matt/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/matt/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/matt/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/matt/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/matt/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/matt/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver statu

ValueError: attempt to get argmax of an empty sequence

In [ ]:
prices.to_csv('/home/mjenior/Desktop/repos/Jenior_FMT_2021/data/consortia_shadow_prices.tsv', sep='\t')

In [ ]:
cdf_prices.to_csv('/home/mjenior/Desktop/repos/Jenior_FMT_2021/data/cdf_shadow_prices.tsv', sep='\t')

In [ ]:
import pandas
import warnings

# Non-exhaustive
def find_essential_exchanges(model, fraction=0.1):

    min_growth = model.slim_optimize() * fraction
    warnings.filterwarnings('ignore')
    ser = minimal_medium(model, min_objective_value=min_growth, minimize_components=True)
    warnings.filterwarnings('default')

    rxns = list(ser.index)
    fluxes = list(ser.values)
    cpds = []
    for x in rxns: cpds.append(model.reactions.get_by_id(x).reactants[0].name)        
    
    media = {'Reaction': rxns, 'Substrate': cpds, 'Units': fluxes}    
    media = pandas.DataFrame(media)
    print('Minimal media components: ' + str(len(media.index)))
    
    return(media)

In [ ]:
consortia_minmedia = find_essential_exchanges(consortia_model)

In [ ]:
Erectale_minmedia = find_essential_exchanges(Erectale, fraction=0.5)
#Afinegoldii_minmedia = find_essential_exchanges(Afinegoldii, fraction=0.5)
#PspCT06_minmedia = find_essential_exchanges(PspCT06, fraction=0.5)
Bvulgatus_minmedia = find_essential_exchanges(Bvulgatus, fraction=0.5)
#Ahadrus_minmedia = find_essential_exchanges(Ahadrus, fraction=0.5)
Fprausnitzii_minmedia = find_essential_exchanges(Fprausnitzii, fraction=0.5)
#CspART55_minmedia = find_essential_exchanges(CspART55, fraction=0.25)
#Bdorei_minmedia = find_essential_exchanges(Bdorei, fraction=0.25)
Bpseudocatenulatum_minmedia = find_essential_exchanges(Bpseudocatenulatum, fraction=0.5)
#Osplanchnicus_minmedia = find_essential_exchanges(Osplanchnicus, fraction=0.25)
#Eeligens_minmedia = find_essential_exchanges(Eeligens, fraction=0.5)
Pdistasonis_minmedia = find_essential_exchanges(Pdistasonis, fraction=0.5)
#Dpiger_minmedia = find_essential_exchanges(Dpiger, fraction=0.25)

In [ ]:
#test = list(Erectale_minmedia['Reaction']) + list(Afinegoldii_minmedia['Reaction']) + list(PspCT06_minmedia['Reaction']) + list(Bvulgatus_minmedia['Reaction']) + list(Ahadrus_minmedia['Reaction']) + list(Fprausnitzii_minmedia['Reaction'])+ list(CspART55_minmedia['Reaction'])+ list(Bdorei_minmedia['Reaction'])+ list(Bpseudocatenulatum_minmedia['Reaction'])+ list(Osplanchnicus_minmedia['Reaction'])+ list(Eeligens_minmedia['Reaction'])+ list(Pdistasonis_minmedia['Reaction'])+ list(Dpiger_minmedia['Reaction'])
test = list(Erectale_minmedia['Reaction']) + list(Bvulgatus_minmedia['Reaction']) + list(Fprausnitzii_minmedia['Reaction']) + list(Bpseudocatenulatum_minmedia['Reaction']) + list(Pdistasonis_minmedia['Reaction'])

In [ ]:
for x in set(test).difference(set(list(consortia_minmedia['Reaction']))):
    print(x, consortia_model.reactions.get_by_id(x).name)

In [ ]:


EX_cpd37231_e = [2-(βGlcNac)-Rib-ol-P]n-[2-(αGlcNac)-Rib-ol-P]-Gro-P-Gro-P-ManNAc-GlcNAc-PP-undecaprenol
EX_cpd00149_e = Cobalt ion
EX_cpd10515_e = ferrous iron
EX_cpd00099_e = Chloride


In [ ]:
media_conditions = {'EX_cpd28247_e': -1000.,
                    'EX_cpd00099_e': -1000.,
                    'EX_cpd00254_e': -1000.,
                    'EX_cpd15540_e': -1000.,
                    'EX_cpd15307_e': -1000.,
                    'EX_cpd00056_e': -1000.,
                    'EX_cpd15240_e': -1000.,
                    'EX_cpd10515_e': -1000.,
                    'EX_cpd15386_e': -1000.,
                    'EX_cpd15545_e': -1000.,
                    'EX_cpd01311_e': -1000.,
                    'EX_cpd15543_e': -1000.,
                    'EX_cpd15309_e': -1000.,
                    'EX_cpd00971_e': -1000.,
                    'EX_cpd00119_e': -1000.,
                    'EX_cpd00063_e': -1000.,
                    'EX_cpd14514_e': -1000.,
                    'EX_cpd00030_e': -1000.,
                    'EX_cpd00205_e': -1000.,
                    'EX_cpd17005_e': -1000.,
                    'EX_cpd00001_e': -1000.,
                    'EX_cpd00067_e': -1000.,
                    'EX_cpd00081_e': -1000.,
                    'EX_cpd17026_e': -1000.,
                    'EX_cpd00007_e': -1000.,
                    'EX_cpd01157_e': -1000.,
                    'EX_cpd00059_e': -1000.,
                    'EX_cpd00041_e': -1000.,
                    'EX_cpd00982_e': -1000.,
                    'EX_cpd00776_e': -1000.,
                    'EX_cpd00078_e': -1000.,
                    'EX_cpd00200_e': -1000.,
                    'EX_cpd15561_e': -1000.,
                    'EX_cpd00106_e': -1000.,
                    'EX_cpd00075_e': -1000.,
                    'EX_cpd01757_e': -1000.,
                    'EX_cpd15298_e': -1000.,
                    'EX_cpd00010_e': -1000.,
                    'EX_cpd00536_e': -1000.,
                    'EX_cpd01018_e': -1000.,
                    'EX_cpd11451_e': -1000.,
                    'EX_cpd01414_e': -1000.,
                    'EX_cpd12556_e': -1000.,
                    'EX_cpd15499_e': -1000.,
                    'EX_cpd00009_e': -1000.,
                    'EX_cpd08374_e': -1000.,
                    'EX_cpd15275_e': -1000.,
                    'EX_cpd08021_e': -1000.,
                    'EX_cpd00033_e': -1000.,
                    'EX_cpd00166_e': -1000.,
                    'EX_cpd17028_e': -1000.,
                    'EX_cpd10516_e': -1000.,
                    'EX_cpd00109_e': -1000.,
                    'EX_cpd00355_e': -1000.,
                    'EX_cpd00137_e': -1000.,
                    'EX_cpd00253_e': -1000.,
                    'EX_cpd15143_e': -1000.,
                    'EX_cpd00149_e': -1000.,
                    'EX_cpd11595_e': -1000.,
                    'EX_cpd00022_e': -1000.,
                    'EX_cpd00070_e': -1000.,
                    'EX_cpd03845_e': -1000.,
                    'EX_cpd30321_e': -1000.,
                    'EX_cpd00158_e': -1000.,
                    'EX_cpd01171_e': -1000.,
                    'EX_cpd00508_e': -1000.,
                    'EX_cpd00314_e': -1000.,
                    'EX_cpd29533_e': -1000.,
                    'EX_cpd00588_e': -1000.,
                    'EX_cpd15498_e': -1000.,
                    'EX_cpd00797_e': -1000.,
                    'EX_cpd00345_e': -1000.,
                    'EX_cpd02229_e': -1000.,
                    'EX_cpd01646_e': -1000.,
                    'EX_cpd15940_e': -1000.,
                    'EX_cpd29700_e': -1000.,
                    'EX_cpd00208_e': -1000.,
                    'EX_cpd00082_e': -1000.,
                    'EX_cpd37231_e': -1000.,
                    'EX_cpd00308_e': -1000.,
                    'EX_cpd11456_e': -1000.,
                    'EX_cpd03495_e': -1000.,
                    'EX_cpd15560_e': -1000.}

rel_abunds = {'Eubacterium_rectale': 0.0769,
              'Alistipes_finegoldii': 0.0769,
              'Parabacteroides_sp._CT06': 0.0769,
              'Bacteroides_vulgatus': 0.0769,
              'Anaerostipes_hadrus': 0.0769,
              'Faecalibacterium_prausnitzii': 0.0769,
              'Coprococcus_sp._ART55': 0.0769,
              'Bacteroides_dorei': 0.0769,
              'Bifidobacterium_pseudocatenulatum': 0.0769,
              'Odoribacter_splanchnicus': 0.0769,
              'Eubacterium_eligens': 0.0769,
              'Parabacteroides_distasonis': 0.0769,
              'Desulfovibrio_piger': 0.0769}

consortia_model = constrain_media_and_abundance(consortia_model, media_conditions, rel_abunds)

In [2]:
iCdR703 = cobra.io.load_json_model('/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR703.json')

FileNotFoundError: [Errno 2] No such file or directory: '/home/mjenior/Desktop/repos/Jenior_Cdifficile_2019/data/reconstructions/iCdR703.json'

In [ ]:
medium = ['cpd00001_e', # water
'cpd00082_e', # D-fructose
'cpd00035_e', # alanine
'cpd00041_e', # aspartic acid
'cpd00023_e', # glutamic acid
'cpd00119_e', # histidine
'cpd00107_e', # leucine
'cpd00060_e', # methionine
'cpd00161_e', # threonine
'cpd00069_e', # tyrosine
'cpd00084_e', # cysteine
'cpd00033_e', # glycine
'cpd00322_e', # isoleucine
'cpd00066_e', # phenylalanine
'cpd00054_e', # serine
'cpd00065_e', # tryptophan
'cpd00156_e', # valine
'cpd00220_e', # riboflavin
'cpd00644_e', # pantothentate(B5)
'cpd00393_e', # folate(B9)
'cpd00133_e', # niacin(PP)
'cpd00263_e', # Pyridoxine
'cpd00104_e', # Biotin
'cpd00149_e', # Cobalt
'cpd00971_e', # sodium
'cpd00099_e', # Chloride
'cpd00205_e', # potassium
'cpd00009_e', # phosphate
'cpd00063_e', # calcium
'cpd00254_e', # magnesium
'cpd10515_e', # Fe2+
'cpd00030_e', # Mn2+
'cpd00242_e', # Carbonate
'cpd00226_e', # hypoxanthine
'cpd01242_e', # thyminose
'cpd00307_e', # cytosine
'cpd00092_e', # uracil
'cpd00117_e', # D-Alanine
'cpd00067_e', # H+
'cpd00567_e', # D-Proline
'cpd00132_e', # L-Asparagine
'cpd00210_e', # Taurine
'cpd00320_e', # D-Aspartate
'cpd03279_e', # Deoxyinosine
'cpd00246_e', # Inosine
'cpd00311_e', # Guanosine
'cpd00367_e', # Cytidine
'cpd00277_e', # Deoxyguanosine
'cpd00182_e', # Adenosine
'cpd00654_e', # Deoxycytidine
'cpd00412_e', # Deoxyuridine
'cpd00438_e', # Deoxyadenosine
'cpd00274_e', # Citrulline
'cpd00186_e', # D-Glutamate
'cpd00637_e', # D-Methionine
'cpd00105_e', # D-Ribose
'cpd00305_e', # Thiamin
'cpd00309_e', # Xanthine
'cpd00098_e', # Choline
'cpd00053_e', # L-Glutamine
'cpd00039_e', # Lysine
'cpd00064_e', # Ornithine
'cpd00051_e', # D-arginine
'cpd00207_e'] # Guanine

medium_exch = set(['EX_' + x for x in medium])
for rxn in iCdR703.exchanges:
    rxn.lower_bound = 0.
    if rxn.id in medium_exch:
        rxn.lower_bound = -1000.